# Cálculo de Diferencia de Diferencias (DifDif)

Este notebook calcula nuevas variables que miden la **diferencia de diferencias** entre Generales y Ballotage.

## Concepto:

Para cada ítem, calculamos:

1. **Diferencia en Generales**: `Dif_Gen = Izq - Der` (en Generales)
2. **Diferencia en Ballotage**: `Dif_Bal = Izq - Der` (en Ballotage)
3. **Diferencia de Diferencias**: `DifDif = Dif_Bal - Dif_Gen`

Esto nos dice **cómo cambió la asimetría izquierda-derecha** entre ambas elecciones.

## Variables calculadas:

- `DifDif_CO_Item_X`: Para Cambio de Opinión
- `DifDif_CT_Item_X`: Para Cambio de Tiempo

Donde X es el número de ítem (3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30)

In [1]:
import pandas as pd
import numpy as np
import os

print("Librerías cargadas exitosamente")

Librerías cargadas exitosamente


## 1. Cargar Datos

In [2]:
# Rutas a los archivos Excel
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames desde Excel
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

print(f"✓ Datos cargados desde Excel:")
print(f"  - Generales: {len(df_Generales)} registros, {len(df_Generales.columns)} columnas")
print(f"  - Ballotage: {len(df_Ballotage)} registros, {len(df_Ballotage.columns)} columnas")

✓ Datos cargados desde Excel:
  - Generales: 2786 registros, 488 columnas
  - Ballotage: 1254 registros, 471 columnas


## 2. Definir Ítems a Procesar

In [3]:
# Ítems progresistas y conservadores
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]

# Todos los ítems
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f"Total de ítems a procesar: {len(Todos_Items)}")
print(f"Ítems: {Todos_Items}")

Total de ítems a procesar: 20
Ítems: [3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30]


## 3. Función para Calcular Diferencia de Diferencias

In [4]:
def Calcular_DifDif_Para_Tipo(df_gen, df_bal, items, tipo):
    """
    Calcula las variables de diferencia de diferencias para un tipo específico (CO o CT).
    
    Parámetros:
    - df_gen: DataFrame de Generales
    - df_bal: DataFrame de Ballotage
    - items: Lista de números de ítems
    - tipo: 'CO' o 'CT'
    
    Retorna:
    - DataFrame con las columnas DifDif calculadas
    """
    
    # DataFrame resultado
    df_resultado = pd.DataFrame()
    
    # Para cada ítem
    for item in items:
        # Nombres de las variables
        var_izq = f'{tipo}_Item_{item}_Izq'
        var_der = f'{tipo}_Item_{item}_Der'
        
        # Verificar que las columnas existen en ambos DataFrames
        if var_izq in df_gen.columns and var_der in df_gen.columns and \
           var_izq in df_bal.columns and var_der in df_bal.columns:
            
            # Paso 1: Diferencia en Generales (Izq - Der)
            Dif_Gen = df_gen[var_izq] - df_gen[var_der]
            
            # Paso 2: Diferencia en Ballotage (Izq - Der)
            Dif_Bal = df_bal[var_izq] - df_bal[var_der]
            
            # Paso 3: Diferencia de Diferencias
            DifDif = Dif_Bal - Dif_Gen
            
            # Nombre de la variable resultado
            nombre_dif_gen = f'Dif_Gen_{tipo}_Item_{item}'
            nombre_dif_bal = f'Dif_Bal_{tipo}_Item_{item}'
            nombre_difdif = f'DifDif_{tipo}_Item_{item}'
            
            # Agregar al DataFrame resultado
            df_resultado[nombre_dif_gen] = Dif_Gen
            df_resultado[nombre_dif_bal] = Dif_Bal
            df_resultado[nombre_difdif] = DifDif
            
        else:
            print(f"⚠️  Variables {var_izq} o {var_der} no encontradas en ambos DataFrames")
    
    return df_resultado

## 4. Calcular DifDif para CO (Cambio de Opinión)

In [5]:
print("Calculando DifDif para CO (Cambio de Opinión)...\n")

df_DifDif_CO = Calcular_DifDif_Para_Tipo(
    df_gen=df_Generales,
    df_bal=df_Ballotage,
    items=Todos_Items,
    tipo='CO'
)

print(f"\n✓ Calculadas {len(df_DifDif_CO.columns)} variables para CO")
print(f"  Total de filas: {len(df_DifDif_CO)}")
print(f"\nPrimeras columnas: {df_DifDif_CO.columns[:6].tolist()}")

Calculando DifDif para CO (Cambio de Opinión)...


✓ Calculadas 60 variables para CO
  Total de filas: 2786

Primeras columnas: ['Dif_Gen_CO_Item_3', 'Dif_Bal_CO_Item_3', 'DifDif_CO_Item_3', 'Dif_Gen_CO_Item_4', 'Dif_Bal_CO_Item_4', 'DifDif_CO_Item_4']


## 5. Calcular DifDif para CT (Cambio de Tiempo)

In [6]:
print("Calculando DifDif para CT (Cambio de Tiempo)...\n")

df_DifDif_CT = Calcular_DifDif_Para_Tipo(
    df_gen=df_Generales,
    df_bal=df_Ballotage,
    items=Todos_Items,
    tipo='CT'
)

print(f"\n✓ Calculadas {len(df_DifDif_CT.columns)} variables para CT")
print(f"  Total de filas: {len(df_DifDif_CT)}")
print(f"\nPrimeras columnas: {df_DifDif_CT.columns[:6].tolist()}")

Calculando DifDif para CT (Cambio de Tiempo)...


✓ Calculadas 60 variables para CT
  Total de filas: 2786

Primeras columnas: ['Dif_Gen_CT_Item_3', 'Dif_Bal_CT_Item_3', 'DifDif_CT_Item_3', 'Dif_Gen_CT_Item_4', 'Dif_Bal_CT_Item_4', 'DifDif_CT_Item_4']


## 6. Crear DataFrame Elecciones Completo

In [7]:
# Combinar ambos DataFrames
df_Elecciones = pd.concat([df_DifDif_CO, df_DifDif_CT], axis=1)

# Agregar información de identificación desde df_Generales
# Asumiendo que ambos DataFrames tienen el mismo número de filas en el mismo orden
columnas_identificacion = ['Categoria_PASO_2023']

# Agregar más columnas si existen
if 'ID' in df_Generales.columns:
    columnas_identificacion.insert(0, 'ID')

for col in columnas_identificacion:
    if col in df_Generales.columns:
        df_Elecciones.insert(0, col, df_Generales[col].values)

print(f"✓ DataFrame df_Elecciones creado exitosamente")
print(f"  Dimensiones: {df_Elecciones.shape}")
print(f"  Total de variables DifDif: {len([col for col in df_Elecciones.columns if col.startswith('DifDif')])}")
print(f"\nColumnas de identificación: {[col for col in df_Elecciones.columns if not col.startswith('Dif')]}")

✓ DataFrame df_Elecciones creado exitosamente
  Dimensiones: (2786, 122)
  Total de variables DifDif: 40

Columnas de identificación: ['Categoria_PASO_2023', 'ID']


## 7. Exploración de los Datos

In [8]:
# Mostrar las primeras filas
print("Primeras 5 filas de df_Elecciones:\n")
print(df_Elecciones.head())

Primeras 5 filas de df_Elecciones:

      Categoria_PASO_2023    ID  Dif_Gen_CO_Item_3  Dif_Bal_CO_Item_3  \
0        Moderate_Right_B  2792                3.0                NaN   
1  Right_Wing_Libertarian  2800                NaN                NaN   
2        Moderate_Right_A  2805                NaN                NaN   
3               Left_Wing  3148                NaN                NaN   
4                No_Apply  3212                NaN                NaN   

   DifDif_CO_Item_3  Dif_Gen_CO_Item_4  Dif_Bal_CO_Item_4  DifDif_CO_Item_4  \
0               NaN                NaN                NaN               NaN   
1               NaN                NaN                NaN               NaN   
2               NaN                0.0                NaN               NaN   
3               NaN                NaN                NaN               NaN   
4               NaN                NaN                NaN               NaN   

   Dif_Gen_CO_Item_5  Dif_Bal_CO_Item_5  ...  DifD

In [9]:
# Estadísticas descriptivas de las variables DifDif_CO
columnas_difdif_co = [col for col in df_Elecciones.columns if col.startswith('DifDif_CO')]

print(f"\nEstadísticas descriptivas de DifDif_CO (primeras 5 variables):\n")
print(df_Elecciones[columnas_difdif_co[:5]].describe())


Estadísticas descriptivas de DifDif_CO (primeras 5 variables):

       DifDif_CO_Item_3  DifDif_CO_Item_4  DifDif_CO_Item_5  DifDif_CO_Item_6  \
count        176.000000        141.000000        167.000000        164.000000   
mean           0.073864          0.113475         -0.119760         -0.006098   
std            1.556626          1.444656          1.491148          2.017552   
min           -4.000000         -5.000000         -4.000000         -8.000000   
25%            0.000000          0.000000         -1.000000          0.000000   
50%            0.000000          0.000000          0.000000          0.000000   
75%            0.000000          0.000000          0.000000          1.000000   
max            7.000000          4.000000          5.000000          4.000000   

       DifDif_CO_Item_7  
count        135.000000  
mean           0.103704  
std            1.763113  
min           -4.000000  
25%            0.000000  
50%            0.000000  
75%            0.000000

In [10]:
# Estadísticas descriptivas de las variables DifDif_CT
columnas_difdif_ct = [col for col in df_Elecciones.columns if col.startswith('DifDif_CT')]

print(f"\nEstadísticas descriptivas de DifDif_CT (primeras 5 variables):\n")
print(df_Elecciones[columnas_difdif_ct[:5]].describe())


Estadísticas descriptivas de DifDif_CT (primeras 5 variables):

       DifDif_CT_Item_3  DifDif_CT_Item_4  DifDif_CT_Item_5  DifDif_CT_Item_6  \
count        176.000000        141.000000        167.000000        164.000000   
mean           0.892293          0.480277          2.574593          0.038692   
std           13.451007          5.991982         14.583042         11.479872   
min          -25.894000        -29.444000        -17.879000        -43.711000   
25%           -2.522000         -2.053000         -2.175500         -4.624000   
50%           -0.404000          0.746000          0.384000         -0.419500   
75%            2.591250          3.431000          3.737000          3.226250   
max          146.585000         16.242000        148.364500         65.332000   

       DifDif_CT_Item_7  
count        135.000000  
mean          -1.764437  
std           10.488542  
min          -63.497000  
25%           -2.790000  
50%           -0.121000  
75%            2.338000

## 8. Verificar Valores Faltantes

In [11]:
# Contar valores faltantes
print("Valores faltantes por variable DifDif_CO:\n")
valores_faltantes_co = df_Elecciones[columnas_difdif_co].isnull().sum()
print(valores_faltantes_co[valores_faltantes_co > 0])
if valores_faltantes_co.sum() == 0:
    print("  ✓ No hay valores faltantes en DifDif_CO")

Valores faltantes por variable DifDif_CO:

DifDif_CO_Item_3     2610
DifDif_CO_Item_4     2645
DifDif_CO_Item_5     2619
DifDif_CO_Item_6     2622
DifDif_CO_Item_7     2651
DifDif_CO_Item_8     2622
DifDif_CO_Item_9     2638
DifDif_CO_Item_10    2629
DifDif_CO_Item_11    2633
DifDif_CO_Item_16    2632
DifDif_CO_Item_19    2631
DifDif_CO_Item_20    2639
DifDif_CO_Item_22    2649
DifDif_CO_Item_23    2630
DifDif_CO_Item_24    2634
DifDif_CO_Item_25    2614
DifDif_CO_Item_27    2626
DifDif_CO_Item_28    2641
DifDif_CO_Item_29    2636
DifDif_CO_Item_30    2633
dtype: int64


In [12]:
print("\nValores faltantes por variable DifDif_CT:\n")
valores_faltantes_ct = df_Elecciones[columnas_difdif_ct].isnull().sum()
print(valores_faltantes_ct[valores_faltantes_ct > 0])
if valores_faltantes_ct.sum() == 0:
    print("  ✓ No hay valores faltantes en DifDif_CT")


Valores faltantes por variable DifDif_CT:

DifDif_CT_Item_3     2610
DifDif_CT_Item_4     2645
DifDif_CT_Item_5     2619
DifDif_CT_Item_6     2622
DifDif_CT_Item_7     2651
DifDif_CT_Item_8     2622
DifDif_CT_Item_9     2638
DifDif_CT_Item_10    2629
DifDif_CT_Item_11    2633
DifDif_CT_Item_16    2632
DifDif_CT_Item_19    2631
DifDif_CT_Item_20    2639
DifDif_CT_Item_22    2649
DifDif_CT_Item_23    2630
DifDif_CT_Item_24    2634
DifDif_CT_Item_25    2614
DifDif_CT_Item_27    2626
DifDif_CT_Item_28    2641
DifDif_CT_Item_29    2636
DifDif_CT_Item_30    2633
dtype: int64


## 9. Distribución por Categoría

In [13]:
# Contar registros por categoría
if 'Categoria_PASO_2023' in df_Elecciones.columns:
    print("\nDistribución de registros por Categoría PASO 2023:\n")
    print(df_Elecciones['Categoria_PASO_2023'].value_counts().sort_index())
    print(f"\nTotal: {df_Elecciones['Categoria_PASO_2023'].value_counts().sum()} registros")


Distribución de registros por Categoría PASO 2023:

Categoria_PASO_2023
Blank                       66
Centre                      84
Left_Wing                  425
Moderate_Right_A           208
Moderate_Right_B           212
No_Apply                   325
No_Response                 24
Progressivism             1257
Right_Wing_Libertarian     185
Name: count, dtype: int64

Total: 2786 registros


## 10. Guardar DataFrame Elecciones

In [14]:
# Crear carpeta si no existe
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar en Excel
Ruta_Salida = os.path.join(Carpeta_Salida, 'df_Elecciones.xlsx')
df_Elecciones.to_excel(Ruta_Salida, index=False)

print(f"\n✅ DataFrame guardado exitosamente en: {Ruta_Salida}")
print(f"   {len(df_Elecciones)} registros × {len(df_Elecciones.columns)} columnas")


✅ DataFrame guardado exitosamente en: c:\Users\Patricio\Documents\Codigo\Python\Investigacion\Tesis\Código\..\Data\Procesados\df_Elecciones.xlsx
   2786 registros × 122 columnas


## 11. Resumen Final

In [15]:
print("="*70)
print("RESUMEN DEL CÁLCULO DE DIFERENCIA DE DIFERENCIAS")
print("="*70)
print(f"\n📊 Variables calculadas:")
print(f"   - Variables DifDif_CO: {len(columnas_difdif_co)}")
print(f"   - Variables DifDif_CT: {len(columnas_difdif_ct)}")
print(f"   - Total variables DifDif: {len(columnas_difdif_co) + len(columnas_difdif_ct)}")

print(f"\n📁 DataFrame df_Elecciones:")
print(f"   - Dimensiones: {df_Elecciones.shape[0]} filas × {df_Elecciones.shape[1]} columnas")
print(f"   - Archivo guardado: df_Elecciones.xlsx")

print(f"\n🎯 Interpretación de DifDif:")
print(f"   - DifDif > 0: La diferencia Izq-Der aumentó en Ballotage")
print(f"   - DifDif < 0: La diferencia Izq-Der disminuyó en Ballotage")
print(f"   - DifDif = 0: No hubo cambio en la asimetría Izq-Der")

print(f"\n✅ Proceso completado exitosamente")
print("="*70)

RESUMEN DEL CÁLCULO DE DIFERENCIA DE DIFERENCIAS

📊 Variables calculadas:
   - Variables DifDif_CO: 20
   - Variables DifDif_CT: 20
   - Total variables DifDif: 40

📁 DataFrame df_Elecciones:
   - Dimensiones: 2786 filas × 122 columnas
   - Archivo guardado: df_Elecciones.xlsx

🎯 Interpretación de DifDif:
   - DifDif > 0: La diferencia Izq-Der aumentó en Ballotage
   - DifDif < 0: La diferencia Izq-Der disminuyó en Ballotage
   - DifDif = 0: No hubo cambio en la asimetría Izq-Der

✅ Proceso completado exitosamente
